In [27]:
import pandas as pd
from collections import Counter
from PIL import Image,ImageDraw,ImageFont
from scipy.stats import spearmanr
import os
import numpy as np

In [28]:
df = pd.read_csv("lime-robustness-all-score-outcome.csv")

In [29]:
df = df[df.nsmp==5000]
df = df[df.nft==5]

In [30]:
df.head()

,officer_id,nsmp,nft,nunq,unq,rand,unit_score,outcome
31,3735,5000,5,5,"{'ts_id_1d_trafficstopswitharrest_sum', 'ocnd_...",no,1.000000,0
73,508,5000,5,7,"{'outemp_id_5y_outsideemploymenthours_sum', 'o...",no,0.988909,1
115,4661,5000,5,5,"{'ts_id_1d_trafficstopswitharrest_sum', 'ocnd_...",no,0.432389,0
157,5743,5000,5,5,"{'ts_id_1d_trafficstopswitharrest_sum', 'ocnd_...",no,0.431597,1
199,3251,5000,5,5,"{'ts_id_1d_trafficstopswitharrest_sum', 'ocnd_...",no,0.413052,0


#### Officer Table

In [31]:
officer_table = df[['officer_id','unit_score','outcome','unq']]

In [32]:
officer_table_groups = officer_table.groupby('officer_id').groups

In [33]:
officer_table_trunc = officer_table[['officer_id','unit_score','outcome']]
officer_table_trunc = officer_table_trunc.groupby("officer_id").agg(np.mean)
officer_table_trunc.reset_index(inplace=True)

In [34]:
featlists = []
for i in officer_table_trunc.index:
    officer_id = officer_table_trunc.officer_id.loc[i]
    idxs = officer_table_groups[officer_id]
    tmp = list(officer_table.loc[idxs,'unq'])
    
    feats = []
    for item in tmp:
        temp = item.rstrip("}")
        temp = temp.lstrip("{")
        feats.append(temp.split(","))
    feats = [item for sublist in feats for item in sublist]
    feats = [item.translate(None," '") for item in feats]
    featlists.append(feats)
    
officer_table_trunc['feats'] = featlists

In [35]:
officer_table_trunc.to_csv("officer_table.csv",index=False) # just so we have it!

In [36]:
feattable = pd.read_csv("limefeat_alphasort.csv")

In [37]:
tmp = pd.DataFrame(columns=list(feattable.feature))

In [38]:
n = len(officer_table_trunc)

In [39]:
for col in tmp.columns:
    tmp[col] = np.repeat(0,n)

In [40]:
tmp.index = officer_table_trunc.officer_id

In [41]:
for i in officer_table_trunc.officer_id:
    feats = officer_table_trunc.feats[officer_table_trunc.officer_id==i].iloc[0]
    for feat in feats:
        tmp[feat].loc[i] = 1

In [42]:
tmp.to_csv("lime_table.csv")

In [43]:
eo = pd.read_csv("explained_officers.csv")

In [44]:
delcol = []
for col in tmp:
    if sum(tmp[col])==0:
        del tmp[col]
        delcol.append(col)

In [45]:
eo.set_index("officer_id",inplace=True)

In [46]:
del eo['as_of_date']

In [47]:
for col in delcol:
    del eo[col]

In [48]:
officer_table_trunc.set_index("officer_id",inplace=True)

In [49]:
tmp['outcome'] = officer_table_trunc.outcome

In [50]:
eo.to_csv("raw.csv")
tmp.to_csv("lime.csv")

In [51]:
eo.outcome.value_counts()

0    53
1    13
Name: outcome, dtype: int64

In [52]:
tmp.outcome.value_counts()

0    53
1    13
Name: outcome, dtype: int64

In [54]:
eo.head()

,ts_id_1d_trafficstopswitharrest_avg,ts_id_1d_trafficstopswitharrest_sum,ocnd_id_dummyofficergender_female_max,ocnd_id_dummyofficergender_unknown_max,ocnd_id_dummyofficergender_male_max,fi_id_1y_fieldinterviewsbyrace_american_indian_sum,fi_id_1y_fieldinterviewsbyrace_black_sum,arrests_id_5y_arrestson_fri_sum,demarrests_id_1y_foreclosures_avg,demarrests_id_5y_foreclosures_avg,...,ir_id_1y_incidentsoftype_complaint_sum,ir_id_1y_incidentsoftype_raid_search_sum,arrests_id_5y_arrestsoftype_ar5_sum,ir_id_5y_allallegations_sum,fi_id_5y_fieldinterviewsbyrace_unknown_sum,demarrests_id_5y_arrests311call_avg,fi_id_5y_fieldinterviewsbyoutcome_none_avg,demarrests_id_1y_violentcrimerate_avg,outemp_id_5y_outsideemploymenthours_sum,outcome
officer_id,,,,,,,,,,,,,,,,,,,,,
4,0.0,0,0,0,1,0,8,17,1.800,1.700000,...,0,0,3,0,1,0.0,0.666667,2.475,0.0,0
30,0.0,0,0,0,1,0,10,8,1.450,0.893750,...,0,0,9,3,0,0.0,0.763158,10.250,0.0,0
191,0.0,0,0,0,1,0,17,50,0.425,0.719608,...,0,0,72,14,0,385.4,0.517241,24.000,0.0,0
508,0.0,0,1,0,0,0,0,3,2.000,2.000000,...,0,24,1,119,85,0.0,1.000000,2.500,0.0,1
566,0.0,0,0,1,0,0,0,0,0.000,0.000000,...,0,0,0,0,0,0.0,0.000000,0.000,0.0,0
